In [244]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Uploading the dataset

In [245]:
df = pd.read_excel('FM_2023_original.xlsx')

In [246]:
df

,Name,Position,Age,Current Ability,Potential Ability,Nationality,Club,League,Country,Corners,...,MC,LW,CAM,RW,ST,Height,Left Foot,Right Foot,Values,Salary
0,Kevin De Bruyne,M/AM RLC,31,189,189,Belgium,Manchester City,Premier League,England,14,...,20,14,20,14,12,181,16,20,347975206,394372
1,Kylian Mbappé,AM/S RL,23,188,197,France,Paris Saint-Germain,Ligue 1,France,13,...,1,19,1,17,20,178,10,20,347975206,1035616
2,Robert Lewandowski,S,33,186,190,Poland,Barcelona,La Liga,Spain,3,...,1,10,12,8,20,185,13,20,347975206,345204
3,Erling Haaland,S,22,185,195,"Norway,England",Manchester City,Premier League,England,7,...,1,1,1,1,20,195,20,11,347975206,394372
4,Mohamed Salah,AM/S RL,30,185,187,Egypt,Liverpool,Premier League,England,12,...,1,17,12,20,19,175,20,8,347975206,405971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,Callum Perry,M/AM L,16,48,140,England,Coventry City,Championship,England,5,...,1,20,1,1,1,174,20,7,235092,203
5366,William Gutierrez Ramirez,D R,16,46,135,England,Reading,Championship,England,3,...,1,1,1,1,1,158,7,20,132370,203
5367,Joe Ashton,D L,16,45,135,England,Burnley,Championship,England,3,...,1,1,1,1,1,181,20,7,124343,203
5368,Adijat Sefer,S,17,45,135,Germany,TSG Hoffenheim,Bundesliga,Germany,3,...,1,1,1,1,20,184,7,20,70999,646


# Data cleaning

## Potential Ability

In [247]:
#resolver potential ability

# Select rows where 'pa' is negative
negative_pa = df['Potential Ability'] < 0

# Replace negative values in 'pa' with the sum of their positive value and 'ca'
df.loc[negative_pa, 'Potential Ability'] = df.loc[negative_pa, 'Potential Ability'].abs() + df.loc[negative_pa, 'Current Ability']

## Wages

In [248]:
# Rename 'Salary' column to 'Weekly wage'
df = df.rename(columns={'Salary': 'Weekly wage'})

# Round 'Weekly wage' to the nearest integer
df['Weekly wage'] = df['Weekly wage'].round().astype(int)

In [249]:
# Create 'Monthly Wage' column by multiplying 'Weekly wage' by 4.3 (average number of weeks in a month)
df['Monthly Wage'] = (df['Weekly wage'] * 4.3).round().astype(int)

# Create 'Annual Wage' column by multiplying 'Weekly wage' by 52 (number of weeks in a year)
df['Annual Wage'] = (df['Weekly wage'] * 52).round().astype(int)

df = df.drop(['Annual Wage','Weekly wage'], axis=1)

## Value (in millions)

In [250]:
df['Value (in millions)'] = df['Values'] / 1000000
df['Value (in millions)'] = df['Value (in millions)'].round(3)
df = df.drop('Values', axis=1)

In [252]:
#Corrigir 'Not for sale'
df.loc[df['Value (in millions)']==347.975, 'Value (in millions)'] = np.nan

In [254]:
# select the columns we want to use for prediction
predictor_cols = ['Current Ability', 'Age', 'Potential Ability', 'Monthly Wage']

# create a new dataframe with just the rows where 'Value (in millions)' is not NaN
df_notna = df.loc[df['Value (in millions)'].notna()]

# fit a linear regression model to the predictor columns and the 'Value (in millions)' column
X = df_notna[predictor_cols]
y = df_notna['Value (in millions)']
lr = LinearRegression()
lr.fit(X, y)

# predict the missing values in the 'Value (in millions)' column
X_missing = df.loc[df['Value (in millions)'].isna(), predictor_cols]
predicted_values = lr.predict(X_missing)

# fill in the missing values in the original dataframe with the predicted values
df.loc[df['Value (in millions)'].isna(), 'Value (in millions)'] = predicted_values

In [255]:
df['Value (in millions)'] = df['Value (in millions)'].round(3)

## Nationality

In [257]:
# Split 'Nationality' column by ',' separator and select first value
df['Nationality'] = df['Nationality'].str.split(',').str[0]

## Main position

In [258]:
#Corrigir posição para ser unica

headers = df.columns[69:79]

for index, row in df.iterrows():
    # get the column with the biggest value for this row
    max_col = max(headers, key=lambda col: row[col])
    # write the header of the column with the biggest value in the 2nd column
    df.at[index, df.columns[1]] = max_col

# Final Dataset

In [259]:
df

,Name,Position,Age,Current Ability,Potential Ability,Nationality,Club,League,Country,Corners,...,MC,LW,CAM,RW,ST,Height,Left Foot,Right Foot,Monthly Wage,Value (in millions)
0,Kevin De Bruyne,MC,31,189,189,Belgium,Manchester City,Premier League,England,14,...,20,14,20,14,12,181,16,20,1695800,138.442
1,Kylian Mbappé,ST,23,188,197,France,Paris Saint-Germain,Ligue 1,France,13,...,1,19,1,17,20,178,10,20,4453149,341.993
2,Robert Lewandowski,ST,33,186,190,Poland,Barcelona,La Liga,Spain,3,...,1,10,12,8,20,185,13,20,1484377,117.932
3,Erling Haaland,ST,22,185,195,Norway,Manchester City,Premier League,England,7,...,1,1,1,1,20,195,20,11,1695800,161.404
4,Mohamed Salah,RW,30,185,187,Egypt,Liverpool,Premier League,England,12,...,1,17,12,20,19,175,20,8,1745675,142.333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5365,Callum Perry,LW,16,48,140,England,Coventry City,Championship,England,5,...,1,20,1,1,1,174,20,7,873,0.235
5366,William Gutierrez Ramirez,RB,16,46,135,England,Reading,Championship,England,3,...,1,1,1,1,1,158,7,20,873,0.132
5367,Joe Ashton,LB,16,45,135,England,Burnley,Championship,England,3,...,1,1,1,1,1,181,20,7,873,0.124
5368,Adijat Sefer,ST,17,45,135,Germany,TSG Hoffenheim,Bundesliga,Germany,3,...,1,1,1,1,20,184,7,20,2778,0.071


# Downloading final data

In [260]:
df.to_excel('FM_2023_Final.xlsx')